# 📝 Exercise M3.01

The goal is to write an exhaustive search to find the best parameters
combination maximizing the model generalization performance.

Here we use a small subset of the Adult Census dataset to make the code
faster to execute. Once your code works on the small subset, try to
change `train_size` to a larger value (e.g. 0.8 for 80% instead of
20%).

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

adult_census = pd.read_csv("../datasets/adult-census.csv")

target_name = "class"
target = adult_census[target_name]
data = adult_census.drop(columns=[target_name, "education-num"])

data_train, data_test, target_train, target_test = train_test_split(
    data, target, train_size=0.8, random_state=42)

In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import OrdinalEncoder

categorical_preprocessor = OrdinalEncoder(handle_unknown="use_encoded_value",
                                          unknown_value=-1)
preprocessor = ColumnTransformer(
    [('cat_preprocessor', categorical_preprocessor,
      selector(dtype_include=object))],
    remainder='passthrough', sparse_threshold=0)

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

model = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", HistGradientBoostingClassifier(random_state=42))
])


Use the previously defined model (called `model`) and using two nested `for`
loops, make a search of the best combinations of the `learning_rate` and
`max_leaf_nodes` parameters. In this regard, you will need to train and test
the model by setting the parameters. The evaluation of the model should be
performed using `cross_val_score` on the training set. We will use the
following parameters search:
- `learning_rate` for the values 0.01, 0.1, 1 and 10. This parameter controls
  the ability of a new tree to correct the error of the previous sequence of
  trees
- `max_leaf_nodes` for the values 3, 10, 30. This parameter controls the
  depth of each tree.

In [3]:
for param in model.get_params():
    print(param)

memory
steps
verbose
preprocessor
classifier
preprocessor__n_jobs
preprocessor__remainder
preprocessor__sparse_threshold
preprocessor__transformer_weights
preprocessor__transformers
preprocessor__verbose
preprocessor__verbose_feature_names_out
preprocessor__cat_preprocessor
preprocessor__cat_preprocessor__categories
preprocessor__cat_preprocessor__dtype
preprocessor__cat_preprocessor__handle_unknown
preprocessor__cat_preprocessor__unknown_value
classifier__categorical_features
classifier__early_stopping
classifier__l2_regularization
classifier__learning_rate
classifier__loss
classifier__max_bins
classifier__max_depth
classifier__max_iter
classifier__max_leaf_nodes
classifier__min_samples_leaf
classifier__monotonic_cst
classifier__n_iter_no_change
classifier__random_state
classifier__scoring
classifier__tol
classifier__validation_fraction
classifier__verbose
classifier__warm_start


In [4]:
# Write your code here.
import numpy as np
from sklearn.model_selection import cross_validate

learning_rates = [0.001 * 10 ** x for x in range(1, 5)]
max_leaf_nodes = [3, 10, 30]

for lr in learning_rates:
    for ml in max_leaf_nodes:
        model.set_params(classifier__learning_rate=lr)
        model.set_params(classifier__max_leaf_nodes=ml)
        cv_result = cross_validate(model, data_train, target_train)
        scores = cv_result["test_score"]
        print(f"Accuracy score via cross-validation with learning_rate={lr} and max_leaf_nodes={ml}:\n"
          f"{scores.mean():.3f} +/- {scores.std():.3f}")

Accuracy score via cross-validation with learning_rate=0.01 and max_leaf_nodes=3:
0.797 +/- 0.002
Accuracy score via cross-validation with learning_rate=0.01 and max_leaf_nodes=10:
0.819 +/- 0.003
Accuracy score via cross-validation with learning_rate=0.01 and max_leaf_nodes=30:
0.848 +/- 0.004
Accuracy score via cross-validation with learning_rate=0.1 and max_leaf_nodes=3:
0.855 +/- 0.003
Accuracy score via cross-validation with learning_rate=0.1 and max_leaf_nodes=10:
0.869 +/- 0.003
Accuracy score via cross-validation with learning_rate=0.1 and max_leaf_nodes=30:
0.871 +/- 0.003
Accuracy score via cross-validation with learning_rate=1.0 and max_leaf_nodes=3:
0.862 +/- 0.007
Accuracy score via cross-validation with learning_rate=1.0 and max_leaf_nodes=10:
0.862 +/- 0.009
Accuracy score via cross-validation with learning_rate=1.0 and max_leaf_nodes=30:
0.859 +/- 0.003
Accuracy score via cross-validation with learning_rate=10.0 and max_leaf_nodes=3:
0.282 +/- 0.005
Accuracy score via c


Now use the test set to score the model using the best parameters
that we found using cross-validation in the training set.

In [5]:
# Write your code here.
model.set_params(classifier__learning_rate=0.1,
                 classifier__max_leaf_nodes=30)

model.fit(data_train, target_train)
test_score = model.score(data_test, target_test)

print(f"Test score after the parameter tuning: {test_score:.3f}")

Test score after the parameter tuning: 0.879
